In [36]:
import requests
import pandas as pd
import json 
from dateutil import parser

In [37]:
API_KEY="cba5fd5258927d19bd27f59d33a08dad-4f3ed1cef55c643a13391fac6767af37"
ACC_NO="101-003-29659100-001"
API="https://api-fxpractice.oanda.com/v3"

In [38]:
session=requests.Session()


In [39]:
session.headers.update({
    "Authorization":f"Bearer {API_KEY}",
    "Content-Type":"application/json"
})

In [40]:
url=f"{API}/accounts/{ACC_NO}/instruments"

Getting the Various Instruments

In [41]:
response=session.get(url,params=None,headers=None,data=None)

In [42]:
data=response.json()

In [43]:
instruments_list=data["instruments"]

In [44]:
key_i=['name','type', 'displayName', 'pipLocation','displayPrecision','tradeUnitsPrecision','marginRate']

In [45]:
 instruments_dict={}
for i in instruments_list:
    key=i["name"]
    instruments_dict[key]={k: i[k] for k in key_i}

In [46]:
with open("../data/instruments.json","w") as f:
    f.write(json.dumps(instruments_dict,indent=2))

In [54]:

def fetch_candles(pair_name, count=10, granularity="H1"):
    url=f'{API}/instruments/{pair_name}/candles'
    params= dict(
        count=count,
        granularity=granularity,
        price="MBA"
    )
    response=session.get(url,params=params, headers=None, data=None)
    data=response.json()
    if response.status_code==200:
        if 'candles' not in data:
            data=[]
        else:
            data=data['candles']
    return response.status_code, data


Turning data into panda Dataframe

In [48]:
prices = ['mid','bid','ask']
ohlc=['o','h','l','c']

In [68]:

def get_candles_df(data):
    final_data=[]

    if len(data)==0:
        return pd.dataframe()
        
    for candle in data:
        if candle['complete']==False:
            continue
    
        new_dict={}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume']=candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] =float(candle[p][o])
        final_data.append(new_dict)
    
    df= pd.DataFrame(final_data)
    final_data=[]
    return df


In [69]:
def create_data_file(pair_name, count=10, granularity="H1"):
    code,data=fetch_candles(pair_name, count,granularity)
    if code!=200:
        print("Failed",pair_name,data)
        return
    if len(data)==0:
        print("No candles",pair_name) 
    candles_df=get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")

In [70]:
create_data_file("EUR_USD")

EUR_USD H1 9 candles, 2024-08-16 00:00:00+00:00 2024-08-16 08:00:00+00:00


In [ ]:
our_curr=["EUR","USD","GBP","JPY","CHF","NZD","CAD","AUD"]
for p1 in our_curr:
    for p2 in our_curr:
        pr=f"{p1}_{p2}"
        if pr in instruments_dict.keys():
            for g in ["H1","H4"]:
                create_data_file(pr,4001,g)

EUR_USD H1 4000 candles, 2023-12-26 16:00:00+00:00 2024-08-16 08:00:00+00:00
EUR_USD H4 4000 candles, 2022-01-21 18:00:00+00:00 2024-08-16 05:00:00+00:00
EUR_GBP H1 4000 candles, 2023-12-26 16:00:00+00:00 2024-08-16 08:00:00+00:00
EUR_GBP H4 4000 candles, 2022-01-21 18:00:00+00:00 2024-08-16 05:00:00+00:00
EUR_JPY H1 4000 candles, 2023-12-26 16:00:00+00:00 2024-08-16 08:00:00+00:00
EUR_JPY H4 4000 candles, 2022-01-21 10:00:00+00:00 2024-08-16 05:00:00+00:00
EUR_CHF H1 4000 candles, 2023-12-26 16:00:00+00:00 2024-08-16 08:00:00+00:00
EUR_CHF H4 4000 candles, 2022-01-21 18:00:00+00:00 2024-08-16 05:00:00+00:00
EUR_NZD H1 4000 candles, 2023-12-26 17:00:00+00:00 2024-08-16 08:00:00+00:00
EUR_NZD H4 4000 candles, 2022-01-24 02:00:00+00:00 2024-08-16 05:00:00+00:00
EUR_CAD H1 4000 candles, 2023-12-26 16:00:00+00:00 2024-08-16 08:00:00+00:00
EUR_CAD H4 4000 candles, 2022-01-21 18:00:00+00:00 2024-08-16 05:00:00+00:00
EUR_AUD H1 4000 candles, 2023-12-26 16:00:00+00:00 2024-08-16 08:00:00+00:00